# Assignment: Build a Prompt-Driven Q&A Chatbot


---

### Objective:
This assignment challenges you to **develop a simple, prompt-driven Q&A chatbot** using a Large Language Model (LLM). You'll learn to design an effective core prompt, manage conversational history, handle various user inputs, and evaluate your chatbot's performance. The goal is to build a basic conversational agent that can answer questions based on its underlying LLM knowledge, guided by your carefully crafted prompts.

---

### Instructions:
1.  **LLM Access**: You'll need access to an LLM API (e.g., Google's Gemini, OpenAI's GPT-4, Anthropic's Claude). For this assignment, we'll assume you're using **Google's Gemini Pro model** via the `google-generativeai` library.
2.  **Environment Setup**: Install the necessary Python library: `pip install google-generativeai`.
3.  **Jupyter Notebook**: All your code, chatbot interactions, observations, and analysis must be documented in this Jupyter Notebook.
4.  **API Key**: Securely handle your API key. It's best practice to load it from an environment variable.
5.  **Iterative Development**: Build your chatbot incrementally, testing each component as you go.
6.  **Analysis**: Critically evaluate your chatbot's responses and identify areas for improvement.

---

## Part 1: Initial Setup and Core Prompt Design
Begin by setting up your API access and crafting the foundational prompt for your Q&A chatbot.

### Task 1.1: API Configuration
Configure the `google-generativeai` library with your API key and initialize the Gemini Pro model.

In [ ]:
import google.generativeai as genai
import os
import time

# --- YOUR API KEY HERE ---
# It's highly recommended to load your API key from an environment variable for security.
# For example: GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
# For this assignment, you can temporarily paste it directly, but be careful not to share your notebook with the key.
GOOGLE_API_KEY = "YOUR_API_KEY_HERE" # Replace with your actual API key

genai.configure(api_key=GOOGLE_API_KEY)

# Initialize the Gemini Pro model
model = genai.GenerativeModel('gemini-pro')

print("API configured and Gemini Pro model loaded!")

### Task 1.2: Crafting the Core Prompt
Design the **system prompt** that defines your chatbot's persona, rules, and purpose. This prompt will be sent with every user query to guide the LLM's responses.

* **Goal**: Create a helpful, concise Q&A assistant.
* **Core Prompt Requirements**:
    * Clearly state its role (e.g., "You are a helpful Q&A assistant.").
    * Instruct it to answer questions directly and concisely.
    * Tell it to avoid conversational fillers like "Hello! How can I help you?" or "Is there anything else?".
    * Specify how it should handle questions it cannot answer (e.g., "If you don't know the answer, state 'I don't have enough information to answer that.'").

* **Your Core Prompt**:
    ```
    [Your Core Prompt String Here]
    ```

In [ ]:
# Define your core prompt (system instruction)
core_prompt = """
You are a helpful and concise Q&A assistant.
Answer questions directly and briefly.
Do not include conversational fillers or greetings.
If you don't have enough information to answer a question, respond with: "I don't have enough information to answer that."
"""

print("Core Prompt Defined:")
print(core_prompt)

### Task 1.3: Testing the Core Prompt
Send a few test questions to the LLM with *only* your core prompt and the user's question. Observe how it behaves.

In [ ]:
def get_response(prompt_text):
    try:
        # For single-turn interactions, you can just use generate_content
        # For multi-turn, you'd use chat.send_message
        response = model.generate_content(prompt_text)
        return response.text.strip()
    except Exception as e:
        return f"Error: {e}"

print("--- Testing Core Prompt ---")

test_question_1 = "What is the capital of France?"
full_prompt_1 = core_prompt + "\nUser: " + test_question_1
print(f"\nUser: {test_question_1}")
print(f"Bot: {get_response(full_prompt_1)}")

test_question_2 = "Explain quantum entanglement in simple terms."
full_prompt_2 = core_prompt + "\nUser: " + test_question_2
print(f"\nUser: {test_question_2}")
print(f"Bot: {get_response(full_prompt_2)}")

test_question_3 = "What is the current population of Mars?"
full_prompt_3 = core_prompt + "\nUser: " + test_question_3
print(f"\nUser: {test_question_3}")
print(f"Bot: {get_response(full_prompt_3)}")

### Analysis for Task 1.3:
* Did the LLM adhere to the persona and rules defined in your core prompt?
* How well did it handle factual questions vs. questions it couldn't answer?
* Identify any unexpected behaviors or areas where the core prompt might need refinement.

---

## Part 2: Building the Chat Loop and History Management
Now, you'll create a basic chat loop and implement a mechanism to pass conversational history to the LLM.

### Task 2.1: Implementing Conversational History
Modify your `get_response` function to use `model.start_chat()` and `chat.send_message()` to maintain conversation history. This allows the LLM to understand context from previous turns.

In [ ]:
def create_chatbot(system_prompt):
    # Initialize a new chat session with the system prompt
    chat = model.start_chat(history=[{'role': 'user', 'parts': [system_prompt]}, {'role': 'model', 'parts': ['OK.']}])
    return chat

def send_message_to_chatbot(chat_session, user_message):
    try:
        response = chat_session.send_message(user_message)
        return response.text.strip()
    except Exception as e:
        return f"Error: {e}"

# Initialize the chatbot with your core prompt
my_chatbot = create_chatbot(core_prompt)

print("--- Testing Chatbot with History ---")

# Test questions to check context
print("\nUser: What is the largest ocean on Earth?")
response_1 = send_message_to_chatbot(my_chatbot, "What is the largest ocean on Earth?")
print(f"Bot: {response_1}")

print("\nUser: And how deep is its deepest point?")
response_2 = send_message_to_chatbot(my_chatbot, "And how deep is its deepest point?")
print(f"Bot: {response_2}")

print("\nUser: What is the highest mountain on land?")
response_3 = send_message_to_chatbot(my_chatbot, "What is the highest mountain on land?")
print(f"Bot: {response_3}")

print("\nUser: Is it taller than the deepest point you mentioned earlier?")
response_4 = send_message_to_chatbot(my_chatbot, "Is it taller than the deepest point you mentioned earlier?")
print(f"Bot: {response_4}")


### Analysis for Task 2.1:
* Did the chatbot successfully use context from previous turns (e.g., understanding "its deepest point" without explicitly naming the ocean again)?
* Are there any instances where context was lost or misinterpreted?
* Discuss the advantages and potential disadvantages of passing full conversation history to the LLM.

### Task 2.2: Building the Interactive Chat Loop
Create a loop that allows a user to continuously interact with your chatbot until they type an exit command (e.g., `quit`, `exit`, `bye`).

In [ ]:
def run_chatbot_conversation(initial_prompt):
    print("\n--- Starting Chatbot Conversation ---")
    print("Type 'quit' or 'exit' to end the conversation.")

    # Re-initialize chatbot for a fresh conversation
    chat = create_chatbot(initial_prompt)

    while True:
        user_input = input("\nUser: ")
        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("Bot: Goodbye!")
            break

        bot_response = send_message_to_chatbot(chat, user_input)
        print(f"Bot: {bot_response}")

# Run the chatbot
# Call this function to start the interactive session:
# run_chatbot_conversation(core_prompt)

# To run it, uncomment the line below and execute the cell.
# run_chatbot_conversation(core_prompt)

print("The interactive chat loop function is defined. Uncomment the last line to run it.")

### Task 2.3: Manual Chatbot Testing and Evaluation
Run your interactive chatbot. Engage in a conversation, asking various types of questions. Document at least **5-7 turns of interaction** (User and Bot messages) and provide a qualitative evaluation of your chatbot's performance.

**Paste Your Interaction Log Here:**
```
User: [Your first question]
Bot: [Bot's response]

User: [Your second question]
Bot: [Bot's response]

...
```

### Analysis for Task 2.3:
* How effective was your chatbot at answering diverse questions?
* Did it maintain its persona and follow the instructions in the core prompt throughout the conversation?
* Identify any instances of:
    * **Hallucination**: Providing incorrect but confidently stated information.
    * **Irrelevant responses**: Not addressing the user's query.
    * **Failure to follow instructions**: (e.g., still using greetings, not stating "I don't have enough information").
    * **Context loss**: Forgetting previous turns.
* Based on this manual testing, what are the top 2-3 areas for improvement?

---

## Part 3: Advanced Prompting and Refinements (Optional/Bonus)
This section is for those who want to explore more advanced techniques to improve their chatbot.

### Task 3.1 (Bonus): Adding Specific Knowledge or Constraints
Modify your core prompt or add an initial context to your `chat.start_chat()` history to provide the chatbot with specific, limited knowledge or additional constraints (e.g., make it a Q&A bot about *only* a specific historical period, or tell it to only answer questions from a provided text). Test its ability to adhere to these new boundaries.

**Example Additions**: "Only answer questions about the Roman Empire."
"Here is a text about renewable energy: [Your Text]. Answer questions ONLY from this text. If the answer is not in the text, say 'I can only answer from the provided text.'"

In [ ]:
# --- Implement your modified chatbot setup here ---
# new_core_prompt = "..."
# new_chatbot = create_chatbot(new_core_prompt)
# run_chatbot_conversation(new_core_prompt) # Remember to uncomment to run

# Provide interaction logs and analysis similar to Task 2.3.

---

## Part 4: Conclusion and Reflection
In a markdown cell, provide a comprehensive summary of your findings and reflections based on this assignment.

* **Effectiveness of Prompt Engineering**: How crucial is prompt engineering in guiding an LLM's behavior for a Q&A chatbot?
* **Challenges in Chatbot Development**: What were the main challenges you faced in building and testing your prompt-driven chatbot?
* **Importance of History**: How significant is maintaining conversation history for a Q&A chatbot, and what are its limitations?
* **Future Improvements**: If you had more time, what specific features or improvements would you add to your chatbot?
* **Ethical Considerations**: What ethical concerns (e.g., misinformation, bias, privacy) might arise from deploying a simple Q&A chatbot like this?

---

### Submission:
* Ensure all code cells have been executed and their outputs (including prints and any interaction logs) are visible.
* All analysis and reflections are clearly written in markdown cells.
* Save your Jupyter Notebook as `[YourName]_Q&A_Chatbot_Assignment.ipynb`.